# Preparar datos y hacer diversos modelos 

Para poder hacer un buen modelo tenemos que seleccionar los parametros correctos además de hacer una limpieza adecuada, eso es lo que haremos. 

In [47]:
import pandas as pd
from sklearn.metrics import  mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import dagshub
import mlflow
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [34]:
df = pd.read_csv('/home/melanie/PycharmProjects/final-exam-pcd2024-autumn/raw_data/data.csv')
#df = df[['complaint_what_happened', 'ticket_classification']]
df

,Unnamed: 0,complaint_what_happened,ticket_classification
0,0,Good morning my name is XXXX XXXX and I apprec...,Debt collection + Credit card debt
1,1,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Credit card or prepaid card + General-purpose ...
2,2,Chase Card was reported on XX/XX/2019. However...,"Credit reporting, credit repair services, or o..."
3,3,"On XX/XX/2018, while trying to book a XXXX XX...","Credit reporting, credit repair services, or o..."
4,4,my grand son give me check for {$1600.00} i de...,Checking or savings account + Checking account
...,...,...,...
18958,18958,My husband passed away. Chase bank put check o...,Checking or savings account + Checking account
18959,18959,After being a Chase Card customer for well ove...,Credit card or prepaid card + General-purpose ...
18960,18960,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",Credit card or prepaid card + General-purpose ...
18961,18961,I am not familiar with XXXX pay and did not un...,Checking or savings account + Checking account


Preprocesamiento del texto para mejorar los modelos.

In [35]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Función para limpiar el texto
def preprocess_text(text):
    text = re.sub(r"XX|xx", "", text)  # Eliminar "XX"
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Mantener solo letras
    text = text.lower()  # Convertir a minúsculas
    tokens = text.split()  # Tokenizar
    tokens = [word for word in tokens if word not in stop_words]  # Eliminar stopwords
    return " ".join(tokens)

# Aplicar preprocesamiento
df['cleaned_text'] = df['complaint_what_happened'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/melanie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Separar los los datos para entrenar los modelos. 

In [37]:
# Separar datos en X (features) y y (target)
X = df['cleaned_text']
y = df['ticket_classification']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Entrenamiento de modelos.

In [39]:
# Pipeline con TF-IDF y Logistic Regression
tfidf_lr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),
    ('clf', LogisticRegression(max_iter=1000))
])

# Entrenamiento y evaluación
tfidf_lr_pipeline.fit(X_train, y_train)
lr_accuracy = tfidf_lr_pipeline.score(X_test, y_test)
print("Logistic Regression Accuracy:", lr_accuracy)

Logistic Regression Accuracy: 0.5839704719219615


In [40]:
# Vectorización TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

# Predicción y evaluación
y_pred_rf = rf_model.predict(X_test_tfidf)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.5599789085156868


In [48]:

# Crear pipeline para SVM
svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # Vectorizador TF-IDF
    ('clf', SVC(kernel='linear', probability=True, random_state=42))  # SVM con kernel lineal
])

# Entrenar el modelo
svm_pipeline.fit(X_train, y_train)

# Predicción y evaluación
y_pred_svm = svm_pipeline.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", svm_accuracy)

SVM Accuracy: 0.5958344318481413


In [50]:
Carga de modelos a dagshub.

SyntaxError: invalid syntax (1846093420.py, line 1)

In [43]:
# Inicializar conexión con DagsHub
dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)

# Configurar el tracking URI para MLflow
TRACKING_URI = mlflow.get_tracking_uri()
mlflow.set_tracking_uri(TRACKING_URI)

# Definir el experimento
mlflow.set_experiment("melanie-michel")

Initialized MLflow to track repo "zapatacc/final-exam-pcd2024-autumn"

Repository zapatacc/final-exam-pcd2024-autumn initialized!

<Experiment: artifact_location='mlflow-artifacts:/02688bbbcea44e41a2c981a9aad1106e', creation_time=1732306000706, experiment_id='43', last_update_time=1732306000706, lifecycle_stage='active', name='melanie-michel', tags={}>

In [44]:
#Registrar el modelo Logistic Regression
with mlflow.start_run(run_name="Logistic Regression Model") as run:
    # Registrar parámetros y métricas
    mlflow.log_param("model", "Logistic Regression")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_metric("accuracy", lr_accuracy)

    # Guardar el modelo
    mlflow.sklearn.log_model(tfidf_lr_pipeline, "logistic_regression_model")

2024/11/22 14:07:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression Model at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/43/runs/349a456f84c04ccb96759c2d69458666
🧪 View experiment at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/43


In [45]:
#registrar el modelo Random Forest
with mlflow.start_run(run_name="Random Forest Model") as run:
    # Registrar parámetros y métricas
    mlflow.log_param("model", "Random Forest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_metric("accuracy", rf_accuracy)

    # Guardar el modelo
    mlflow.sklearn.log_model(rf_model, "random_forest_model")

2024/11/22 14:08:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest Model at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/43/runs/4b3c02e377cc4fcca3a3cac258f9659c
🧪 View experiment at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/43


In [49]:
# Registrar el modelo SVM en MLflow
with mlflow.start_run(run_name="SVM Model") as run:
    # Registrar parámetros y métricas
    mlflow.log_param("model", "Support Vector Machine")
    mlflow.log_param("kernel", "linear")
    mlflow.log_metric("accuracy", svm_accuracy)

    # Guardar el modelo
    mlflow.sklearn.log_model(
        svm_pipeline, 
        "svm_model"
    )

2024/11/22 14:41:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run SVM Model at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/43/runs/e2d52469966b4756b55a8ac12879656c
🧪 View experiment at: https://dagshub.com/zapatacc/final-exam-pcd2024-autumn.mlflow/#/experiments/43
